<a href="https://colab.research.google.com/github/thayammal/thayammal/blob/main/IC_skincancer_VGG16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Mount with drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#@title Include all dependencies
import os
import cv2
from PIL import Image
import tensorflow as tf
import keras
from keras import backend as K
from keras.models import load_model
from tensorflow.keras.utils import img_to_array
#from keras.preprocessing.image import img_to_array
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
#Dec 21
# Try with generalized method for importing number of classes
import pathlib
import numpy as np
data_dir = pathlib.Path('/content/drive/MyDrive/IC_CombineDB/Image_Classification/dataset/skin_plant_disease/train/')
train_dir = data_dir
valid_dir = pathlib.Path('/content/drive/MyDrive/IC_CombineDB/Image_Classification/dataset/skin_plant_disease/validation/')
CLASS_NAMES = np.array([item.name for item in data_dir.glob('*') if item.name != "LICENSE.txt"])
CLASS_NAMES

array(['malignant', 'benign', 'Anthracnose', 'nutrient_deficiency',
       'Alternaria_Alternata', 'Bacterial_Blight', 'Cercospora_Leaf_Spot',
       'Healthy_Leaves'], dtype='<U20')

In [ ]:
import random
all_images = list(data_dir.glob('*/*'))
all_images = [str(path) for path in all_images]
random.shuffle(all_images)
data_size=len(all_images)
print(data_size)

3972


In [ ]:
!mkdir ~/.keras
!mkdir ~/.keras/models
!cp ../input/keras-pretrained-models/*notop* ~/.keras/models/
!cp ../input/keras-pretrained-models/imagenet_class_index.json ~/.keras/models/

mkdir: cannot create directory ‘/root/.keras’: File exists
cp: cannot stat '../input/keras-pretrained-models/*notop*': No such file or directory
cp: cannot stat '../input/keras-pretrained-models/imagenet_class_index.json': No such file or directory


In [ ]:
num_classes=len(CLASS_NAMES)
# importing the libraries
from keras.models import Model
from keras.layers import Flatten, Dense
from keras.applications import VGG16
#from keras.preprocessing import image

IMAGE_SIZE = [150, 150]  # we will keep the image size as (64,64). You can increase the size for better results. 

# loading the weights of VGG16 without the top layer. These weights are trained on Imagenet dataset.
vgg = VGG16(input_shape = IMAGE_SIZE + [3], weights = 'imagenet', include_top = False)  # input_shape = (64,64,3) as required by VGG

# this will exclude the initial layers from training phase as there are already been trained.
for layer in vgg.layers:
    layer.trainable = False

x = Flatten()(vgg.output)
#x = Dense(128, activation = 'relu')(x)   # we can add a new fully connected layer but it will increase the execution time.
x = Dense(num_classes, activation = 'softmax')(x)  # adding the output layer with softmax function as this is a multi label classification problem.

model = Model(inputs = vgg.input, outputs = x)

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

58889256/58889256 [==============================] - 1s 0us/step


In [ ]:
# Image Augmentation
IMAGE_SIZE = [150, 150]
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import preprocess_input

training_datagen = ImageDataGenerator(
                                    rescale=1./255,   # all pixel values will be between 0 an 1
                                    shear_range=0.2, 
                                    zoom_range=0.2,
                                    horizontal_flip=True,
                                    preprocessing_function=preprocess_input)

validation_datagen = ImageDataGenerator(rescale = 1./255, preprocessing_function=preprocess_input)

training_generator = training_datagen.flow_from_directory(train_dir, target_size = IMAGE_SIZE, batch_size = 200, class_mode = 'categorical')
validation_generator = validation_datagen.flow_from_directory(valid_dir, target_size = IMAGE_SIZE, batch_size = 200, class_mode = 'categorical')

Found 3972 images belonging to 8 classes.
Found 620 images belonging to 8 classes.


In [ ]:
#number of classes in the dataset
training_generator.class_indices

{'Alternaria_Alternata': 0,
 'Anthracnose': 1,
 'Bacterial_Blight': 2,
 'Cercospora_Leaf_Spot': 3,
 'Healthy_Leaves': 4,
 'benign': 5,
 'malignant': 6,
 'nutrient_deficiency': 7}

In [ ]:
#Fit the Model 
history = model.fit(training_generator,
                    epochs=10, 
                    steps_per_epoch=2,
                    validation_steps=2,
                    validation_data=validation_generator)

Epoch 1/10
2/2 [==============================] - 240s 181s/step - loss: 2.1029 - accuracy: 0.0950 - val_loss: 1.1946 - val_accuracy: 0.5625
Epoch 2/10
2/2 [==============================] - 76s 43s/step - loss: 1.0312 - accuracy: 0.6075 - val_loss: 0.8244 - val_accuracy: 0.7175
Epoch 3/10
2/2 [==============================] - 59s 27s/step - loss: 0.7009 - accuracy: 0.7419 - val_loss: 0.7194 - val_accuracy: 0.7500
Epoch 4/10
2/2 [==============================] - 57s 29s/step - loss: 0.5554 - accuracy: 0.8025 - val_loss: 0.6626 - val_accuracy: 0.7625
Epoch 5/10
2/2 [==============================] - 52s 27s/step - loss: 0.5731 - accuracy: 0.8250 - val_loss: 0.6484 - val_accuracy: 0.7400
Epoch 6/10
2/2 [==============================] - 44s 24s/step - loss: 0.5065 - accuracy: 0.8300 - val_loss: 0.5920 - val_accuracy: 0.7975
Epoch 7/10
2/2 [==============================] - 36s 19s/step - loss: 0.4368 - accuracy: 0.8525 - val_loss: 0.6055 - val_accuracy: 0.8000
Epoch 8/10
2/2 [=========

In [ ]:
history = model.fit_generator(training_generator,
                   steps_per_epoch = 10000,  # this should be equal to total number of images in training set. But to speed up the execution, I am only using 10000 images. Change this for better results. 
                   epochs = 1,  # change this for better results
                   validation_data = validation_generator,
                   validation_steps = 3000)  # this should be equal to total number of images in validation set.

In [ ]:
channels = 3
IMG_SIZE = 150

dataset = np.ndarray(shape=(len(all_images), IMG_SIZE, IMG_SIZE,channels),
                     dtype=np.float32)
i = 0
for _file in dataset:#all_images:
  image = cv2.imread(_file,1)
  #cv2_imshow(image)
  image = cv2.resize(image,(IMG_SIZE,IMG_SIZE))
  dataset[i] = image
  i += 1
print("All images to array!")

TypeError: ignored

In [ ]:
dataset=dataset/255.
dataset.min(),dataset.max(),dataset.shape

(0.0, 1.0, (4022, 150, 150, 3))

In [ ]:
mean = dataset.mean(axis=(0,1,2))
std = dataset.std(axis=(0,1,2))
print(mean, std)

[0.18539384 0.18539384 0.18539384] [0.3040935  0.3452565  0.42589602]


In [ ]:
dataset[..., 0] -= mean[0]
dataset[..., 1] -= mean[1]
dataset[..., 2] -= mean[2]

In [ ]:
dataset.min(axis=(0,1,2)),dataset.max(axis=(0,1,2))

(array([-0.18539384, -0.18539384, -0.18539384], dtype=float32),
 array([0.8146062, 0.8146062, 0.8146062], dtype=float32))

In [ ]:
dataset[..., 0] /= std[0]
dataset[..., 1] /= std[1]
dataset[..., 2] /= std[2]

In [ ]:
dataset.min(axis=(0,1,2)),dataset.max(axis=(0,1,2))

(array([-0.6096606 , -0.5369742 , -0.43530306], dtype=float32),
 array([2.6788015, 2.3594232, 1.912688 ], dtype=float32))

In [ ]:
train_test_split=(int)(data_size*0.2)

x_train=all_images[train_test_split:]
x_test=all_images[:train_test_split]

y_train=CLASS_NAMES[train_test_split:]
y_test=CLASS_NAMES[:train_test_split]

In [ ]:
BATCH_SIZE = 32

def _parse_data(x,y):
  image = tf.read_file(x)
  image = tf.image.decode_jpeg(image, channels=3)
  image = tf.cast(image, tf.float32)
  image = (image/127.5) - 1
  image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
 
  return image,y

def _input_fn(x,y):
  ds=tf.data.Dataset.from_tensor_slices((x,y))
  ds=ds.map(_parse_data)
  ds=ds.shuffle(buffer_size=data_size)
  
  
  ds = ds.repeat()
  
  ds = ds.batch(BATCH_SIZE)
  
  ds = ds.prefetch(buffer_size=AUTOTUNE)
  
  return ds
  

tf.keras.applications.vgg16.preprocess_input

<function keras.applications.vgg16.preprocess_input(x, data_format=None)>

In [ ]:
IMG_SHAPE = (IMG_SIZE, IMG_SIZE, 3)
VGG16_MODEL=tf.keras.applications.VGG16(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

58889256/58889256 [==============================] - 0s 0us/step


In [ ]:
VGG16_MODEL.trainable=False
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
prediction_layer = tf.keras.layers.Dense(len(CLASS_NAMES),activation='softmax')

In [ ]:
model = tf.keras.Sequential([
  VGG16_MODEL,
  global_average_layer,
  prediction_layer
])

In [ ]:
model.compile(optimizer='adam', loss=tf.keras.losses.sparse_categorical_crossentropy, metrics=['acc'])


In [ ]:
# Image Augmentation

from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import preprocess_input

training_datagen = ImageDataGenerator(
                                    rescale=1./255,   # all pixel values will be between 0 an 1
                                    shear_range=0.2, 
                                    zoom_range=0.2,
                                    horizontal_flip=True,
                                    preprocessing_function=preprocess_input)

validation_datagen = ImageDataGenerator(rescale = 1./255, preprocessing_function=preprocess_input)

training_generator = training_datagen.flow_from_directory(training_dir, target_size = IMAGE_SIZE, batch_size = 200, class_mode = 'categorical')
validation_generator = validation_datagen.flow_from_directory(validation_dir, target_size = IMAGE_SIZE, batch_size = 200, class_mode = 'categorical')

In [ ]:
train_ds=[x_train,y_train]
validation_ds=[x_test,y_test]

In [ ]:
history = model.fit(train_ds,
                    epochs=10, 
                    steps_per_epoch=2,
                    validation_steps=2,
                    validation_data=validation_ds)

ValueError: ignored

In [ ]:
train_ds=_input_fn(x_train,y_train)
validation_ds=_input_fn(x_test,y_test)

ValueError: ignored

In [ ]:
model.compile(optimizer=tf.train.AdamOptimizer(), 
              loss=tf.keras.losses.sparse_categorical_crossentropy,
              metrics=["accuracy"])

AttributeError: ignored

In [ ]:
channels = 3
IMG_SIZE=150
#path = os.path.join(data_dir, train, all_images)
                    
dataset = np.ndarray(shape=(len(all_images), IMG_SIZE, IMG_SIZE,channels),
                     dtype=np.float32)
i = 0
for _file in all_images:
  image = cv2.imread(_file,1)
  if image is None:
    print('Wrong path:', data_dir)
  else:
    image = cv2.resize(image, dsize=(128,128))
  #cv2_imshow(image)
  image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
  dataset[i] = image
  i += 1
print("All images to array!")


All images to array!


In [ ]:
#@title Set path of training , testing and validating data directories
base_dir = '/content/drive/MyDrive/IC_CombineDB/Image Classification/dataset/skin_plant_disease'
train_dir = '/content/drive/MyDrive/IC_CombineDB/Image Classification/dataset/skin_plant_disease/train'
train_benign_dir = '/content/drive/MyDrive/IC_CombineDB/Image Classification/dataset/skin_plant_disease/train/benign'
train_malign_dir = '/content/drive/MyDrive/IC_CombineDB/Image Classification/dataset/skin_plant_disease/train/malignant'
test_dir = '/content/drive/MyDrive/IC_CombineDB/Image Classification/dataset/skin_plant_disease/test'
test_benign_dir = '/content/drive/MyDrive/IC_CombineDB/Image Classification/dataset/skin_plant_disease/test/benign'
test_malign_dir = '/content/drive/MyDrive/IC_CombineDB/Image Classification/dataset/skin_plant_disease/test/malignant'
valid_dir = '/content/drive/MyDrive/IC_CombineDB/Image Classification/dataset/skin_plant_disease/validation'
valid_benign_dir = '/content/drive/MyDrive/IC_CombineDB/Image Classification/dataset/skin_plant_disease/validation/benign'
valid_malign_dir = '/content/drive/MyDrive/IC_CombineDB/Image Classification/dataset/skin_plant_disease/validation/malignant'

In [ ]:
num_benign_train = len(os.listdir(train_benign_dir))
num_malignant_train = len(os.listdir(train_malign_dir))
num_benign_validaition = len(os.listdir(valid_benign_dir))
num_malignant_validation= len(os.listdir(valid_malign_dir))
num_benign_test = len(os.listdir(test_benign_dir))
num_malignant_test= len(os.listdir(test_malign_dir))


In [ ]:
print("Total Training Benign Images",num_benign_train)
print("Total Training Malignant Images",num_malignant_train)
print("--")
print("Total validation Benign Images",num_benign_validaition)
print("Total validation Malignant Images",num_malignant_validation)
print("--")
print("Total Test Benign Images", num_benign_test)
print("Total Test Malignant Images",num_malignant_test)
total_train = num_benign_train+num_malignant_train
total_validation = num_benign_validaition+num_malignant_validation
total_test = num_benign_test+num_malignant_test
print("Total Training Images",total_train)
print("--")
print("Total Validation Images",total_validation)
print("--")
print("Total Testing Images",total_test)

Total Training Benign Images 1140
Total Training Malignant Images 937
--
Total validation Benign Images 300
Total validation Malignant Images 260
--
Total Test Benign Images 360
Total Test Malignant Images 300
Total Training Images 2077
--
Total Validation Images 560
--
Total Testing Images 660


In [ ]:
IMG_SHAPE  = 224
batch_size = 32

In [ ]:
image_gen_train = ImageDataGenerator(rescale = 1./255)
train_data_gen = image_gen_train.flow_from_directory(batch_size = batch_size,
directory = train_dir,
shuffle= True,
target_size = (IMG_SHAPE,IMG_SHAPE),
class_mode = 'binary')
image_generator_validation = ImageDataGenerator(rescale=1./255)
val_data_gen = image_generator_validation.flow_from_directory(batch_size=batch_size,
directory=valid_dir,
target_size=(IMG_SHAPE, IMG_SHAPE),
class_mode='binary')
image_gen_test = ImageDataGenerator(rescale=1./255)
test_data_gen = image_gen_test.flow_from_directory(batch_size=batch_size,
directory=test_dir,
target_size=(IMG_SHAPE, IMG_SHAPE),
class_mode='binary')

Found 2827 images belonging to 7 classes.
Found 570 images belonging to 7 classes.
Found 667 images belonging to 7 classes.


In [ ]:
pre_trained_model = tf.keras.applications.VGG16(input_shape=(224, 224, 3), include_top=False, weights="imagenet")

58889256/58889256 [==============================] - 2s 0us/step


If Training performance is not satisfied then need to do the following optimization methods

•	Mini-batch gradient descent
•	Exponentially weighted average
•	Bias correction
•	Gradient descent with momentum
•	RMS prop
•	Adam
•	Learning rate
•	 The problem of local optima


In [ ]:
for layer in pre_trained_model.layers:
  print(layer.name)
  layer.trainable = False

NameError: ignored

In [ ]:
last_layer = pre_trained_model.get_layer('block5_pool')
last_output = last_layer.output
x = tf.keras.layers.GlobalMaxPooling2D()(last_output)
x = tf.keras.layers.Dense(512, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(7, activation='sigmoid')(x)

In [ ]:
model = tf.keras.Model(pre_trained_model.input, x)


In [ ]:
model.compile(optimizer='adam', loss=tf.keras.losses.sparse_categorical_crossentropy, metrics=['acc'])

In [ ]:
vgg_classifier = model.fit(train_data_gen,
steps_per_epoch=(total_train//batch_size),
epochs = 30,
validation_data=val_data_gen,
validation_steps=(total_validation//batch_size),
batch_size = batch_size,
verbose = 1)

Epoch 1/30
64/64 [==============================] - 1008s 16s/step - loss: 1.0678 - acc: 0.5220 - val_loss: 0.6905 - val_acc: 0.6985
Epoch 2/30
64/64 [==============================] - 210s 3s/step - loss: 0.6261 - acc: 0.6831 - val_loss: 0.5707 - val_acc: 0.7665
Epoch 3/30
64/64 [==============================] - 67s 1s/step - loss: 0.5145 - acc: 0.7749 - val_loss: 0.5313 - val_acc: 0.7978
Epoch 4/30
64/64 [==============================] - 37s 563ms/step - loss: 0.4738 - acc: 0.8125 - val_loss: 0.7628 - val_acc: 0.5974
Epoch 5/30
64/64 [==============================] - 30s 475ms/step - loss: 0.4601 - acc: 0.7953 - val_loss: 0.5629 - val_acc: 0.7500
Epoch 6/30
64/64 [==============================] - 30s 461ms/step - loss: 0.4192 - acc: 0.8194 - val_loss: 0.6970 - val_acc: 0.5754
Epoch 7/30
64/64 [==============================] - 30s 459ms/step - loss: 0.4486 - acc: 0.7918 - val_loss: 0.4820 - val_acc: 0.7647
Epoch 8/30
64/64 [==============================] - 29s 459ms/step - loss:

In [ ]:
result = model.evaluate(test_data_gen,batch_size=batch_size)
print("test_loss, test accuracy",result)

21/21 [==============================] - 170s 8s/step - loss: 0.4583 - acc: 0.7556
test_loss, test accuracy [0.4582826495170593, 0.7556222081184387]


In [ ]:
model_json = model.to_json()
with open("/content/drive/MyDrive/Image Classification/VGG_Skin_Classifier.json", "w") as json_file:
json_file.write(model_json)
model.save("/content/drive/MyDrive/Image Classification/VGG_Skin_Classifier.h5")
print("Saved model to disk")
model.save_weights("/content/drive/MyDrive/Image Classification/VGG_Skin.h5")

In [ ]:
from google.colab import files
uploaded = files.upload()



Saving 03EczemaAreolae1.jpg to 03EczemaAreolae1 (1).jpg


In [ ]:
class_names = train_data_gen.class_names
print(class_names)

AttributeError: ignored

In [ ]:
import imageio
unknownim=imageio.imread('03EczemaAreolae1 (1).jpg')

FileNotFoundError: ignored

In [ ]:
import numpy as np
#img = tf.keras.utils.load_img(unknownim, target_size=(224, 224))
img_array = tf.keras.utils.img_to_array(unknownim)
img_array = tf.expand_dims(img_array, 0) # Create a batch

predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])
print(100*np.max(score))

#print("This image most likely belongs to {} with a {:.2f} percent confidence.".format(class_names[np.argmax(score)], 100 * np.max(score)))

NameError: ignored